In [9]:
import numpy as np
import math

In [3]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold         # Threshold value for the split
        self.left = left                   # Left child node
        self.right = right                 # Right child node
        self.value = value                 # Value if leaf node (class prediction)

In [4]:
class HDDT:
    def __init__(self, min_samples_split=2, max_depth=100):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        
    def fit(self, X, y):
        self.root = self._grow_tree(X, y)
        
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        
        # Stopping criteria
        if (depth >= self.max_depth or n_samples < self.min_samples_split or n_classes == 1):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # Find best split
        best_feature, best_threshold = self._best_split(X, y)
        
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # Split the dataset
        left_idxs = X[:, best_feature] <= best_threshold
        right_idxs = X[:, best_feature] > best_threshold
        
        left = self._grow_tree(X[left_idxs], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs], y[right_idxs], depth+1)
        
        return Node(best_feature, best_threshold, left, right)
    
    def _best_split(self, X, y):
        best_hellinger = -1
        best_feature = None
        best_threshold = None
        
        n_features = X.shape[1]
        
        for feature_idx in range(n_features):
            thresholds = np.unique(X[:, feature_idx])
            
            for threshold in thresholds:
                # Split the data
                left_idxs = X[:, feature_idx] <= threshold
                right_idxs = X[:, feature_idx] > threshold
                
                if len(y[left_idxs]) == 0 or len(y[right_idxs]) == 0:
                    continue
                
                # Calculate Hellinger distance
                current_hellinger = self._hellinger_distance(y[left_idxs], y[right_idxs])
                
                if current_hellinger > best_hellinger:
                    best_hellinger = current_hellinger
                    best_feature = feature_idx
                    best_threshold = threshold
                    
        return best_feature, best_threshold
    
    def _hellinger_distance(self, y_left, y_right):
        # Calculate class distributions for left and right splits
        left_counts = np.bincount(y_left)
        right_counts = np.bincount(y_right)
        
        # Normalize to get probabilities
        p_left = left_counts / len(y_left)
        p_right = right_counts / len(y_right)
        
        # Make sure both arrays have the same length (for binary classification)
        max_length = max(len(p_left), len(p_right))
        p_left = np.pad(p_left, (0, max_length - len(p_left)), 'constant')
        p_right = np.pad(p_right, (0, max_length - len(p_right)), 'constant')
        
        # Calculate Hellinger distance
        hellinger = np.sqrt(np.sum((np.sqrt(p_left) - np.sqrt(p_right))**2)) / math.sqrt(2)
        
        return hellinger
    
    def _most_common_label(self, y):
        counts = np.bincount(y)
        return np.argmax(counts)
    
    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        
        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

# Multiclass to Binary class conversion:

In [10]:
def convert_to_binary(y, target_class):
    y_binary = np.where(y == target_class, 1, 0)  # 1 for target class, 0 for others -> OvA
    return y_binary

# use the code:

In [5]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=10, n_classes=3, 
                          weights=[0.6, 0.1, 0.3], random_state=42)

In [6]:
# Train HDDT
hddt = HDDT(min_samples_split=5, max_depth=5)
hddt.fit(X, y)

In [7]:
# Make predictions
predictions = hddt.predict(X)

In [8]:
# Evaluate
from sklearn.metrics import classification_report

print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       897
           1       1.00      0.36      0.53       103

    accuracy                           0.93      1000
   macro avg       0.97      0.68      0.75      1000
weighted avg       0.94      0.93      0.92      1000

